# Optimizing A Simple DNN Training Program Using Nsight Systems

## 5.1 Data Transfers between Host and GPU
Any communication between the host and GPU devices usually happens on a PCIe link which is very slow, so it is important that we optimize any data transfers between the host and the GPU.

## 5.2 Analyze the Profile
Let's analyze the data transfers between host and GPU in the report (firstOptimization.nsys-rep) from the first optimization step. Open the report in the Nsight Systems GUI. Expand the `CUDA device row` by clicking on the tiny triangle in front of it. Expand the `All Streams` row to select the `Memory` row and right-click to choose "Show in Events View" option as shown below.

<img src=images/ShowInEventsView.jpg>

This populates the "Events View" window with the memory operations listed in chronlogical order. Click on the 'Duration' column header to sort the table in the Events View by duration so that the longest memory operation shows up first. Right-click on the first entry in the table and select "Show Current on Timeline" as illustrated below.

<img src=images/EventsView.jpg>

This zooms into the event on the timeline and the teal highlights help you find the CUDA API call, `cudaMemcpyAsync`, that initiated the memory operation on the GPU (see the image below). Note that you might have to zoom out a bit and scroll up to find the CUDA API call on the CPU thread.

<img src=images/CudaMemcpyAsyncUsingPageableMemory.jpg>

We notice the following from the timeline:
- All Host-to-Device (HtoD) memcopies are using pageable memory which is a) slower and b) causes the `cudaMemcpyAsync` API call on the CPU thread to block until the operation completes on the GPU.
- The longest memcpy operation takes ~790 microseconds to complete on the GPU.
- The CUDA API call (`cudaMemcpyAsync`) corresponding to the longest memcpy operation is almost 1ms long.

## 5.3 Optimize the Code to Use Pinned Memory
Host (CPU) memory allocations are pageable by default. The GPU cannot access data directly from pageable host memory. When a data transfer is invoked from pageable host memory to device memory, the CUDA driver must first allocate a temporary page-locked (or “pinned”) host array, copy the host data to the pinned array, and then transfer the data from the pinned array to device memory. The pinned memory is used as a staging area for transfers from the host to the device. By directly allocating our host data in pinned memory, we can avoid this extra step and its overhead. See blog [post](https://developer.nvidia.com/blog/how-optimize-data-transfers-cuda-cc/) for more details.

<img src=images/PageableVsPinned.jpg width=50%>

The settings used for the data loader [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) in our program rely on the default value of `pin_memory: False`. Execute the cell below to see the code change made to use pinned memory.

In [ ]:
!diff -U4 --color=always application/main_opt1.py application/main_opt2.py

## 5.4 Profile Again to Verify Optimization
Let's profile again by executing the cell given below to verify if our code change addresses the problem with host-to-device memory transfers.

In [ ]:
!nsys profile --trace cuda,osrt,nvtx \
--capture-range cudaProfilerApi \
--gpu-metrics-device=all \
--output /dli/task/nsys/secondOptimization \
--force-overwrite true \
python3 /dli/task/nsys/application/main_opt2.py

Open the report (secondOptimization.nsys-rep) in the GUI. Similar to how we navigated the timeline in section 5.2, expand the `CUDA device row` and the `All Streams` row under it. Select the `Memory` row and right-click to choose "Show in Events View". Sort the table in the Events View by duration to get the longest memory operation to show up first. Right-click on the topmost event to select "Show current on timeline". You should end up with a view as shown below.

<img src=images/Optimization2.jpg>

In the profile collected after optimization, we observe that
- All HtoD memcopies now use pinned memory
- The longest memcpy is now only 511 micro-seconds compared to 790 micro-seconds before optimization
- The `cudaMemcpyAsync` API call corresponding to the longest memcpy is now only ~20 microseconds.

Now that we have addressed a bottleneck with memory transfers, let's identify the next performance bottleneck in [step 06](06_tensor_core_util.ipynb)